In [1]:
from utils.processing import Processing
import mne

In [3]:
import numpy as np

In [4]:
test_clean = np.load("RNN_model/data/Test_clean.npy")
test_noisy = np.load("RNN_model/data/Test_noisy.npy")